In [103]:
import urllib.request as req
from urllib.parse import urlparse, urlsplit
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sqlite3
import re
pd.options.display.float_format = '{:,.1f}'.format

In [17]:
url = 'https://price.cetizen.com/'
res = req.Request(url)
html = req.urlopen(res).read()
soup = BeautifulSoup(html, 'html.parser')

In [221]:
wireless = {'wireless_1[]':'S', 'wireless_2[]':'K', 'wireless_3[]':'L', 'wireless_1,2[]':'S,K',
            'wireless_1,3[]':'S,L', 'wireless_2,3[]':'K,L', 'wireless_1,2,3[]':'S,K,L', 'wireless_7[]':'SELF'}

def getInfo(tag):
    tag2 = tag.find_all('li', {'style':re.compile('^float:left')})
    pno = urlparse(tag2[0].a['href']).query.split('&')[1].split('=')[1]
    name = tag2[0].text
    model = tag2[1].text
    price = tag2[2].text
    return pno, name, model, price

def getInfoWireless(wireless):
    rstList = []
    tag = soup.find_all('div', {'name':wireless[0]})
    for i in range(len(tag)):
        rstList.append([wireless[1], *getInfo(tag[i])])
    df = pd.DataFrame(rstList, columns={'wireless','pno','name','model','price'})
    return df

def getInfoAll():
    rstList = []
    for wl in wireless.items():        
        rstList.append(getInfoWireless(wl))
    df = pd.concat(rstList)
    return df

In [222]:
df = getInfoAll()